# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_df = pd.read_csv("../output_data/cities.csv")
weather_df

,Date,Country,City,Max Temperature,Cloudiness,Humidity,Wind Speed,Latitude,Longitude
0,1637624587,CL,Punta Arenas,46.51,75,49,19.57,-53.1500,-70.9167
1,1637624776,PF,Rikitea,75.67,0,67,11.95,-23.1203,-134.9692
2,1637625025,TO,Vaini,82.56,20,83,9.22,-21.2000,-175.2000
3,1637624781,ZA,Hermanus,63.90,90,74,11.12,-34.4187,19.2345
4,1637624868,NZ,Mataura,60.48,47,48,4.12,-46.1927,168.8643
...,...,...,...,...,...,...,...,...,...
495,1637625355,BR,Piaçabuçu,78.62,67,81,11.92,-10.4056,-36.4344
496,1637625355,RU,Lev Tolstoy,20.55,96,88,7.38,53.2134,39.4460
497,1637625356,JP,Kaseda-shirakame,52.14,99,68,14.43,31.4167,130.3167
498,1637625357,ID,Bandar,72.79,99,94,2.37,2.0500,99.7500


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
gmaps.configure(api_key = g_key)
locations = weather_df[["Latitude", "Longitude"]]
humidity = weather_df["Humidity"]

In [5]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [16]:
max_temp = 80
min_temp = 70
cloudiness = 10
wind_speed = 10
humidity = 40

best_cities = weather_df.loc[(weather_df["Max Temperature"] <= max_temp) & \
                             (weather_df["Max Temperature"] >= min_temp) & \
                             (weather_df["Cloudiness"] <= cloudiness) & \
                             (weather_df["Humidity"] <= humidity) & \
                             (weather_df["Wind Speed"] <= wind_speed)]
best_cities.count()
best_cities

,Date,Country,City,Max Temperature,Cloudiness,Humidity,Wind Speed,Latitude,Longitude
222,1637625171,NG,Yola,79.47,3,28,1.83,9.2000,12.4833
252,1637624780,US,Pacific Grove,78.91,1,11,6.91,36.6177,-121.9166
330,1637624684,US,Saint George,72.07,1,21,3.44,37.1041,-113.5841
376,1637625275,NE,Arlit,74.07,0,20,4.41,18.7369,7.3853
416,1637625302,CM,Kousséri,75.16,0,25,4.61,12.0769,15.0306
480,1637625122,US,Salinas,79.32,1,14,5.75,36.6777,-121.6555


In [17]:
best_cities_clean = best_cities.dropna()
best_cities_clean

,Date,Country,City,Max Temperature,Cloudiness,Humidity,Wind Speed,Latitude,Longitude
222,1637625171,NG,Yola,79.47,3,28,1.83,9.2000,12.4833
252,1637624780,US,Pacific Grove,78.91,1,11,6.91,36.6177,-121.9166
330,1637624684,US,Saint George,72.07,1,21,3.44,37.1041,-113.5841
376,1637625275,NE,Arlit,74.07,0,20,4.41,18.7369,7.3853
416,1637625302,CM,Kousséri,75.16,0,25,4.61,12.0769,15.0306
480,1637625122,US,Salinas,79.32,1,14,5.75,36.6777,-121.6555


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [18]:
hotel_df = best_cities_clean[["City", "Country", "Max Temperature", "Cloudiness", "Humidity", "Wind Speed", "Latitude", "Longitude"]].copy()

In [19]:
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Max Temperature,Cloudiness,Humidity,Wind Speed,Latitude,Longitude,Hotel Name
222,Yola,NG,79.47,3,28,1.83,9.2000,12.4833,
252,Pacific Grove,US,78.91,1,11,6.91,36.6177,-121.9166,
330,Saint George,US,72.07,1,21,3.44,37.1041,-113.5841,
376,Arlit,NE,74.07,0,20,4.41,18.7369,7.3853,
416,Kousséri,CM,75.16,0,25,4.61,12.0769,15.0306,
480,Salinas,US,79.32,1,14,5.75,36.6777,-121.6555,


In [22]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "radius":5000,
    "type":"lodging",
    "key": g_key
}
for index, row in hotel_df.iterrows():
    lat = row["Latitude"]
    lng = row["Longitude"]
    
    params["location"] = f"{lat},{lng}"
    hotels = requests.get(base_url, params=params).json()
    
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        print(f"hotel : {hotels['results'][0]['name']}")
    except (IndexError):
        print("Hotel Unavailable.. Skip")

hotel : Muna Hotel
hotel : Lovers Point Inn
hotel : Best Western Plus Abbey Inn
hotel : Case de Passage De la Prefecture
hotel : Le Pilotis
hotel : Laurel Inn & Conference Center


In [23]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [37]:
# Add marker layer ontop of heat map
fig = gmaps.figure(map_type="HYBRID")
markers = gmaps.marker_layer(locations,
    info_box_content = hotel_info)

fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))